1. +gsp_persons.birth_year
2. +gsp_persons.birth_region_id
3. +gsp_persons.living_region_id
4. -doesLive_birth_region ( 0,1 )
5. +gsp_persons.nationality_id
6. +gsp_persons.sex_id
7. -mib_orders 3 types -> ( jarayonda, tugallangan, yo'q )
8. -work (ishli yoki yo'q)
9. -zags 3 types -> ( turmush qurgan, turmush qurmagan, ajrashgan )
10. -ma'lumoti ( oliy, o'rta, unknown )
11. -isFatherDefendant (otasining pinppsi iiv_idcard.father_pinpp dan olinadi va defendantlar ichida bor yoki yo'qligi tekshiriladi)
12. -mashinalari soni (YHXX.tech_passport)
13. -jinoyat ko'p sodir bo'ladigan joyga bog'liqligi
14. -payment_history from payment_history
15. +cadastre.neighborhood_id 
16. -isviolent(oldin zo'ravonlik qilgan yoki yo'qligi, agar pinppi iv_violence bo'lsa-1)
17. -

In [24]:
import pandas as pd
import numpy as np
import clickhouse_connect
import bisect
import random
from typing import List, Tuple, Optional

In [25]:
df_schema = pd.read_excel("metadata_schema_adm_big_datav1.xlsx")
df_def_schema = pd.read_excel("DEFENDANTS_SCHEMA.xlsx")
print(len(df_schema['table_name'].unique()))
print(len(df_def_schema['TABLE_NAME'].unique()))

44
85


## FUNCTIONS

In [41]:
def get_pinpp_normal_with_limit(limit=100_000):
    host = "play.clickhouse.com" 
    port = 443
    user = "explorer" 
    password = ""
    query = f"""
SELECT pinpp  FROM gsp_persons limit {limit}
    """
    client = clickhouse_connect.get_client(
        host=host,
        port=port,
        user=user,
        password=password,
        secure=True
    )
    query = query
    result = client.query(query)
    client.close()
    

    return result.result_rows

In [39]:
def get_all_pinpp_of_defendant():
    host = "play.clickhouse.com" 
    port = 443
    user = "explorer"
    password = ""

    query = """SELECT defendant_pinfl  FROM defendants
WHERE toDate(defendant_provision_date) >= '2022-03-31'
ORDER BY defendant_provision_date DESC
"""
    
    client = clickhouse_connect.get_client(host=host, port=port, user=user, password=password)
    result = client.query(query)
    client.close()

    return result.result_rows

all_defendants_pinpp -> list 

limited_defendants_pinpp -> all_defendants_pinpp[:1000]

In [28]:
def get_pinpp_of_normal_humans(not_get_pinfl_list, limit=100000):
    host = "play.clickhouse.com" 
    port = 443
    user = "explorer" 
    password = ""
    excluded_pinpps = ", ".join([f"'{pinpp}'" for pinpp in not_get_pinfl_list])
    
    query = f"""
    SELECT OriginAirportID
    FROM ontime
    WHERE OriginAirportID NOT IN ({excluded_pinpps})
    LIMIT {limit}
    """

    client = clickhouse_connect.get_client(
        host=host,
        port=port,
        user=user,
        password=password,
        secure=True 
    )
    
    result = client.query(query)
    client.close() 
    return result.result_rows

# result=get_pinpp_of_normal_humans(pinfl_list_to_check, limit=100000)

# df_normal=pd.DataFrame(result, columns=['pinfl'])
# df_normal['jinoyatchimi']=0



In [ ]:
#### Bu ikki funksiya birinchi listdagi va ikkinchi listda yo'q ma'lumotlarni qaytaradi
#for pinpp -> list1 = limited normal people pinpp, list2 = all defendants pinpp

# def check_isUnique(list1, list2):
#     return [item for item in list1 if item not in list2]


def check_isUnique_with_bisect(list1, list2):
    list2.sort()
    result = []
    for item in list1:
        pos = bisect.bisect_left(list2, item)
        if pos == len(list2) or list2[pos] != item:
            result.append(item)

    return result

list1 = list(range(1, 100_000))
list2 = list(range(80_000, 120_000))

# result = check_isUnique_with_bisect(list1, list2)
# print(len(result))

In [45]:
import time
start_time = time.time()
check_isUnique_with_bisect(list1, list2)
end_time = time.time()
print(f"{end_time - start_time}.")

0.04415488243103027.


In [51]:
def get_data_by_pinpp(pinpp_list, table_name, column_name):
    config = {
        "host": "play.clickhouse.com",
        "port": 443,
        "user": "explorer",
        "password": "",
        "secure": True
    }

    with clickhouse_connect.get_client(**config) as client:
        # Split the pinpp_list into manageable chunks to avoid query size limits
        chunk_size = 1000
        chunks = [
            pinpp_list[i:i + chunk_size] 
            for i in range(0, len(pinpp_list), chunk_size)
        ]

        result = []

        for chunk in chunks:
            placeholders = ", ".join([f"'{pinpp}'" for pinpp in chunk])
            query = f"""
                SELECT {column_name}
                FROM {table_name}
                WHERE pinpp IN ({placeholders})
            """

            partial_result = client.query(query)
            result.extend(row[0] for row in partial_result.result_rows)

        return result

# EXTRACTING

In [16]:
#Asosiy dataframe, barcha ma'lumotlar to'planadi
df_main = pd.DataFrame()

### PINPP and is defendant

In [ ]:
#Barcha jinoyatchilarning pinpp ma'lumotlari
all_defendants_pinpp = get_all_pinpp_of_defendant()

#100k ta normal insonlar pinpp ma'lumotlari
normal_people_pinpp_notready = get_pinpp_normal_with_limit(100_000)

normal_people_pinpp = check_isUnique_with_bisect(normal_people_pinpp_notready, all_defendants_pinpp)
defendants_limited = all_defendants_pinpp[:1000]

df_normal = pd.DataFrame(normal_people_pinpp, columns=['pinpp'])
df_normal['isdefendant'] = 0
df_defendants = pd.DataFrame(defendants_limited, columns=['pinpp'])
df_defendants['isdefendant'] = 1

df_main = pd.concat([df_normal, df_defendants], ignore_index=True)

### GSP data

In [ ]:
birth_year = get_data_by_pinpp(df_main['pinpp'].tolist(), 'gsp_persons', 'birth_year')
region_id_birth = get_data_by_pinpp(df_main['pinpp'].tolist(), 'gsp_persons', 'region_id')
region_id_living = get_data_by_pinpp(df_main['pinpp'].tolist(), 'gsp_persons', 'living_region_id')
nationality_id = get_data_by_pinpp(df_main['pinpp'].tolist(), 'gsp_person', 'nationality_id')
sex_id = get_data_by_pinpp(df_main['pinpp'].tolist(), 'gsp_person', "sex_id")
neighbourhood_id = get_data_by_pinpp(df_main['pinpp'].tolist(), 'cadastre', 'neighborhood_id')

df_main[{ 'birth_year': birth_year, 'region_id_birth': region_id_birth, 'region_id_living': region_id_living, 'nationality_id':nationality_id, "sex_id":sex_id, "neighbourhood_id":neighbourhood_id}]